In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.compat.v2 as tf2
import tensorflow_datasets as tfds
import datetime

In [2]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST), 
    with_info=True, as_supervised=True)

In [5]:
encoder = info.features['text'].encoder
buffer_size = 1000
batch_size = 10
train_batches = train_data.shuffle(buffer_size).padded_batch(batch_size,drop_remainder=True)
test_batches = test_data.shuffle(buffer_size).padded_batch(batch_size,drop_remainder=True)
#encoder.subwords[:20]

In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 16),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
])
print(model.summary())
log_dir = "/home/manju/code/ML/src/manju_tensorflow_examples/logs/fit/" \
 + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + 'word_embedding_rnn_emb_16_lstm_16_lstm_16_d_16_d_1_adam_lr_0001'
tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir)

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

history = model.fit(train_batches, epochs=10,
                    validation_data=test_batches, 
                    validation_steps=30,
                    callbacks=[tb_callback])

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, None, 16)          130960    
_________________________________________________________________
lstm_13 (LSTM)               (None, None, 16)          2112      
_________________________________________________________________
dense_20 (Dense)             (None, None, 16)          272       
_________________________________________________________________
dense_21 (Dense)             (None, None, 1)           17        
Total params: 133,361
Trainable params: 133,361
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
390/390 [==============================] - 165s 418ms/step - loss: 0.6931 - accuracy: 0.4986 - val_loss: 0.6927 - val_accuracy: 0.5250
Epoch 2/10
390/390 [==============================] - 167s 427ms/step - loss: 0.6926 - accurac

In [21]:
test_loss, test_acc = model.evaluate(test_batches)

390/390 [==============================] - 68s 175ms/step - loss: 0.4149 - accuracy: 0.8633
